In [ ]:
#@title Installation der Softwarepakete
%%capture
#Install libraries
!nvidia-smi
!pip install diffusers
!pip install transformers scipy ftfy accelerate

from diffusers import StableDiffusionPipeline
import torch

#Start the model, prepare it to generate
model_id = "XpucT/Deliberate"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda")

## Nachfolgend kannst du der KI Anweisungen geben wie Sie dein Wunschbild zu zeichnen hat.

### Hinweis 1:
****
Gib deinen *Prompt* möglichst in, mit Komma voneinander getrennten,Stichworten an.

**Beispiel:** "superhero, time-travel, teenager, cool"

### Hinweis 2:
****
Dieses Modell funktioniert am besten wenn die Prompts auf Englisch geschrieben werden. Falls du Hilfe brauchst, melde dich einfach!

### Hinweis 3: Prompt Beispiele
****
1. [superpower*], teenager girl, super power, drawing, artwork, fantasy hero character portrait *(z.B. Firestarter, Plant Whisperer, Wind Rider usw.)
2. steampunk city, christmas, sunset, photorealism, from above
3. japan street, night, raining, moonlight, high detail, anime style
4. mystical forest, vibrant colors, towering trees, magical creatures
5. dramatic battle between elemental forces, with fire, water, earth, and air colliding in a mesmerizing display of power and energy
6. [game*] game styled character, less realistic, stunning background *(z.B. League of Legends, GTA 5, Fortnite usw.)

###Hinweis 4:
Der "**Positive Prompt**" sagt der KI was es in dem Bild generieren soll, der "**Negative Prompt**" sagt was es *nicht* generieren soll.

Mit die "**Orientierungsskala**" sagst du wie sehr sich das Modell an deine Prompts halten soll. Ist der Faktor sehr hoch, folgt das Modell deinen Anweisungen sehr genau, ist er sehr niedrig, wird es verrückter.

Da das Bildgenerierung mittels KI ist ein schrittweiser Prozess, bei dem die KI unterwegs lernt und sich verbessert, um das bestmögliche Bild zu erstellen, die "**Anzahl_Inferenzschritte**" bestimmt die Anzahl dieser Schritte.

Probiere verschiedene Kombinationen aus, bis du das passende Bild gefunden hast!

In [ ]:
# @title Führe diese Zelle aus, bis dir eines der Bilder in der Grundstruktur gefällt!

#Set default parameters
Positiver_Prompt = "plant whisperer, teenager girl, super power, drawing, artwork, fantasy hero character portrait *(z.B. Firestarter, Plant Whisperer, Wind Rider usw.)" #@param {type:"string"}
Negativer_Prompt = "" #@param {type:"string"}
Orientierungsskala = 6.5 #@param {type:"slider", min:3, max: 12, step: 0.5}
Anzahl_Inferenzschritte = 75 #@param {type:"slider", min:5, max: 150, step: 5}

#Set a complexe prompt together
positive_quality_insurance_keywords = "High resolution, Precise focus, Well-defined details, Clean composition, Consistent lighting"
negative_quality_insurance_keywords = "duplicate, bad face, bad eyes, bad hands, bad anatomy, missing fingers, cropped, mutated hands, extra fingers, black and white, lowres, error, cropped, worst quality, low quality, jpeg artifacts, out of frame, watermark, signature"

num_images = 4
concatenated_prompt = [f"{Positiver_Prompt}, {positive_quality_insurance_keywords}"] * num_images
concatenated_negative_prompt = [f"{Negativer_Prompt}, {negative_quality_insurance_keywords}"] * num_images

#Create an 2x2 grid, so 4 pictures are created simultaneously
from PIL import Image

def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size

    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid


height = pipe.unet.config.sample_size * pipe.vae_scale_factor
width = pipe.unet.config.sample_size * pipe.vae_scale_factor
device = "cuda"
generator = torch.Generator(device=device)

#Prepare pipeline, grid and unique seeds
latents = None
seeds = []
for _ in range(num_images):
    # Get a new random seed, store it and use it as the generator state
    seed = generator.seed()
    seeds.append(seed)
    generator = generator.manual_seed(seed)

    image_latents = torch.randn(
        (1, pipe.unet.in_channels, height // 8, width // 8),
        generator = generator,
        device = device
    )
    latents = image_latents if latents is None else torch.cat((latents, image_latents))

# latents should have shape (4, 4, 64, 64) in this case
# latents.shape

#[concatenated_prompt] * num_images,
#[concatenated_negative_prompt] * num_images,

#Generate and show images
with torch.autocast(device):
    images = pipe(
        prompt=concatenated_prompt,
        negative_prompt=concatenated_negative_prompt,
        num_inference_steps=Anzahl_Inferenzschritte,
        guidance_scale=Orientierungsskala,
        latents = latents
    )["images"]

image_grid(images, 2, 2)

## Bildanpassung

In der nächsten Codezelle kannst du das Bild auswählen, welches dir am besten gefallen hat und "Positive" und "Negative" Ergänzungen in deinen Prompts vornehmen.

In [ ]:
# @title Wähle das Bild aus, welches dir am besten gefallen hat und mache Änderungen!

#Choose a seed to work on
Bild = 1 # @param {type:"number"})
picture_minus_one = Bild - 1

seed = seeds[picture_minus_one]   # Second one
#seed

generator.manual_seed(seed)

height = 512
width = 512

#Prepare a pipeline
latents = torch.randn(
    (1, pipe.unet.in_channels, height // 8, width // 8),
    generator = generator,
    device = device
)

#Set default parameters
Positive_Prompt_Ergänzung = "" # @param {type:"string"}
Negative_Prompt_Ergänzung = "" # @param {type:"string"}
Orientierungsskala = 6.5 #@param {type:"slider", min:3, max: 12, step: 0.5}
Anzahl_Inferenzschritte = 50 #@param {type:"slider", min:5, max: 150, step: 5}

#Generate and show images
with torch.autocast("cuda"):
    image = pipe(
        prompt = [f"{Positiver_Prompt}, {Positive_Prompt_Ergänzung}, {positive_quality_insurance_keywords}"],
        negative_prompt = [f"{Negativer_Prompt}, {Negative_Prompt_Ergänzung}, {negative_quality_insurance_keywords}"],
        guidance_scale=Orientierungsskala,
        num_inference_steps=Anzahl_Inferenzschritte,
        latents = latents,
    )["images"]

image[0]